# Using OOI data in the cloud with Pangeo
In this workshop we will use [Pangeo's](http://pangeo.io/) [Binderhub](https://binderhub.readthedocs.io/en/latest/) to do some science with OOI data in the cloud together. Since we are working today on the Binderhub our work will be ephemeral but if you would like to continue working with Pangeo and OOI in the cloud, please join the [OOICloud](https://www.ooicloud.org/) GitHub organization and continue working at [ooi.pangeo.io](ooi.pangeo.io).

Some key packages we will use are [Erddapy](https://ioos.github.io/erddapy/), [Pandas](https://pandas.pydata.org/docs/) and [Xarray](http://xarray.pydata.org/en/stable/) to explore some data from a bottom pressure recorder at Axial Seamount along with an earthquake catalog generated by William Wilcock.

We will then look at how to use Erddapy to search multiple ERDDAP servers at once, and if we have time we will explore [Dask](https://docs.dask.org/en/latest/) and [Dask Delayed](https://docs.dask.org/en/latest/delayed.html) functions to parallelize a data analysis workflow in the cloud.

Further information on using Python to analyze Earth science datasets can be found in the book [Earth and Environmental Data Science](https://earth-env-data-science.github.io/intro) which I use to teach Research Computing in the Earth Sciences.

## Bottom pressure data at Axial Seamount
Let's find some OOI data using the new [OOI Data Explorer](https://dataexplorer.oceanobservatories.org/) and use Erdappy to load these data into Xarray and plot a smoothed representation of the bottom pressure at Axial Seamount using [hvplot](https://hvplot.holoviz.org/).

### First import some required packages

In [ ]:
import pandas as pd
import xarray as xr
import hvplot.xarray
import hvplot.pandas

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (14, 8)

### Next find some data on the OOI Data Explorer

Link to new OOI Data Explorer: https://dataexplorer.oceanobservatories.org/

In [ ]:
from erddapy import ERDDAP

In [ ]:
server = 'http://erddap.dataexplorer.oceanobservatories.org/erddap'
protocol = 'tabledap'

e = ERDDAP(
    server=server,
    protocol=protocol
)

In [ ]:
e.dataset_id = 'ooi-rs03ccal-mj03f-05-botpta301'

In [ ]:
e.get_info_url()

In [ ]:
info_df = pd.read_csv(e.get_info_url(response='csv'))

In [ ]:
info_df.head()

In [ ]:
info_df[info_df['Row Type']=='variable']

In [ ]:
e.variables = ['time', 'botpres']

In [ ]:
e.constraints = {
    'time>=': '2014-01-01T00:00:00Z',
    'time<=': '2015-12-31T00:00:00Z',
}

In [ ]:
botpt_df = e.to_pandas()

In [ ]:
botpt_df.head()

In [ ]:
botpt_df.plot();

In [ ]:
botpt_ds = e.to_xarray()
botpt_ds

In [ ]:
botpt_ds = e.to_xarray(drop_variables=['station', 'rowSize'])
botpt_ds

In [ ]:
botpt_ds = botpt_ds.swap_dims({'obs': 'time'})
botpt_ds

In [ ]:
botpt_ds= botpt_ds.drop_dims('timeseries')
botpt_ds

In [ ]:
import hvplot.xarray

In [ ]:
botpt_ds.botpres.hvplot()

In [ ]:
botpt_ds.botpres.sel(time=slice('2015-05-01', '2015-07-01')).hvplot()

In [ ]:
botpt_rolling = botpt_ds.rolling(time=60*24*7, min_periods=60*24*7, center=True).mean()

In [ ]:
botpt_rolling.botpres.hvplot()

## Earthquake catalog from the OOI seismic array at Axial Seamount
Here we parse and plot Axial Seamount earthquake catalog data from [William Wilcock's near-real-time automated earthquake location system](http://axial.ocean.washington.edu/). The data we will use is a text file in they HYPO71 output format located here: http://axial.ocean.washington.edu/hypo71.dat.

In [ ]:
eqs_url = 'hypo71.dat'

In [ ]:
col_names = ['ymd', 'hm', 's', 'lat_deg', 'lat_min', 'lon_deg', 'lon_min',
        'depth', 'MW', 'NWR', 'GAP', 'DMIN',  'RMS',  'ERH', 'ERZ', 'ID', 'PMom', 'SMom']

In [ ]:
eqs = pd.read_csv(eqs_url, sep = '\s+', header=0, names=col_names)

In [ ]:
eqs.head()

In [ ]:
from datetime import datetime
def parse_hypo_date(ymd, hm, s):
    hour = int(hm.zfill(4)[0:2])
    minute = int(hm.zfill(4)[2:])
    second = float(s)
    if second == 60:
        second = 0
        minute += 1
    if minute == 60:
        minute=0
        hour +=1
    eq_date_str = ('%s%02.0f%02.0f%05.2f' % (ymd, hour, minute, second))
    return datetime.strptime(eq_date_str, '%Y%m%d%H%M%S.%f')

In [ ]:
eqs = pd.read_csv(eqs_url, sep = '\s+', header=0, names=col_names, parse_dates=[[0,1,2]],
                 date_parser=parse_hypo_date)

In [ ]:
eqs.head()

In [ ]:
eqs['lat'] = eqs.lat_deg+eqs.lat_min/60
eqs['lon'] = -(eqs.lon_deg+eqs.lon_min/60)

In [ ]:
eqs.head()

In [ ]:
eqs.rename(columns={'ymd_hm_s':'time', 'MW':'mw'}, inplace=True)

In [ ]:
eqs.set_index('time', inplace=True)

In [ ]:
eqs = eqs[['lat','lon','depth','mw']]

In [ ]:
eqs.head()

In [ ]:
eqs.to_pickle('hypo71.pkl')

### Load earthquake catalog pickle file

In [ ]:
eqs_df = pd.read_pickle('hypo71.pkl')

In [ ]:
eqs_df.head()

In [ ]:
eqs_ds = eqs_df.to_xarray()

In [ ]:
eqs_ds

In [ ]:
eqs_ds = eqs_ds.set_coords(['lat', 'lon'])

In [ ]:
eqs_ds

In [ ]:
eqs_ds.mw.hvplot.scatter(x='time', datashade=True)

https://xarray.pydata.org/en/stable/user-guide/combining.html

In [ ]:
all_ds = xr.merge([eqs_ds, botpt_ds])

In [ ]:
all_ds

In [ ]:
all_ds.mw.plot(marker='.', linestyle='', markersize=1)

In [ ]:
all_ds.mw.hvplot.scatter(datashade=True)

In [ ]:
all_ds.mw.hvplot.scatter(datashade=True, x='time')

### Daily Counts

In [ ]:
daily_count = all_ds.mw.resample(time='1D').count()

In [ ]:
daily_count

In [ ]:
daily_count.plot()

In [ ]:
fig, ax1 = plt.subplots()
ax1.bar(daily_count.to_series()['2015'].index, daily_count.to_series()['2015'].values, width=4)
ax2 = ax1.twinx()
ax2.plot(botpt_rolling.botpres.to_series()['2015'], color='cyan')

### Mapping eq data
Let's make some maps just because we can.

In [ ]:
import cartopy.crs as ccrs
import cartopy
import numpy as np

In [ ]:
caldera = pd.read_csv('caldera.csv')

In [ ]:
caldera.head()

In [ ]:
now = pd.Timestamp('now')
eqs_sub = eqs[now-pd.Timedelta(weeks=2):]

In [ ]:
ax = plt.axes(projection = ccrs.Robinson(central_longitude=-130))
ax.plot(caldera.lon, caldera.lat, transform=ccrs.Geodetic())
ax.gridlines()

sc = ax.scatter(eqs_sub.lon, eqs_sub.lat, c=eqs_sub.mw, transform=ccrs.PlateCarree(),
               cmap='magma', edgecolor='k', s=40)
plt.colorbar(sc, label='magnitude')

## OOI Seafloor Camera Data
Now let's look at some video data from the [OOI Seafloor Camera](https://oceanobservatories.org/instrument-class/camhd/) system deployed at Axial Volcano on the Juan de Fuca Ridge. We will make use of the [Pycamhd](https://github.com/tjcrone/pycamhd) library, which can be used to extract frames from the ProRes encoded Quicktime files. These data are hosted on Microsoft's [Azure Open Datasets](https://azure.microsoft.com/en-us/services/open-datasets/).

In [ ]:
dbcamhd_url = 'https://ooiopendata.blob.core.windows.net/camhd/dbcamhd.json'

In [ ]:
def show_image(frame_number):
    plt.rc('figure', figsize=(12, 6))
    plt.rcParams.update({'font.size': 8})
    frame = camhd.get_frame(mov.url, frame_number)
    fig, ax = plt.subplots();
    im1 = ax.imshow(frame);
    plt.yticks(np.arange(0,1081,270))
    plt.xticks(np.arange(0,1921,480))
    plt.title('Deployment: %s    File: %s    Frame: %s' % (mov.deployment, mov['name'], frame_number));